## 初始化

In [1]:
from qlib.contrib.data.handler import Alpha158
from qlib.data.dataset import DatasetH
from qlib.data.dataset.loader import QlibDataLoader
def qlib_init():
    import qlib 
    from qlib.config import REG_CN
    provider_uri = "~/.qlib/qlib_data/cn_data"  # target_dir
    qlib.init(provider_uri=provider_uri, region=REG_CN)
def load_alpha158_handler(start_time,end_time,valid_end,market):
    from qlib.contrib.data.handler import Alpha158
    config = {
        "start_time": start_time,
        "end_time": end_time,
        "fit_start_time": start_time,
        "fit_end_time": valid_end,
        "instruments": market,
    }
    return Alpha158(**config)
def load_lightGBM():
    from qlib.contrib.model.gbdt import LGBModel
    config = {
            "loss": "mse",
            "colsample_bytree": 0.8879,
            "learning_rate": 0.0421,
            "subsample": 0.8789,
            "lambda_l1": 205.6999,
            "lambda_l2": 580.9768,
            "max_depth": 8,
            "num_leaves": 210,
            "num_threads": 20,
        }
    model = LGBModel(**config) # model
    return model
  

In [2]:
qlib_init()# qlib 初始化

[13448:MainThread](2023-03-15 09:32:54,324) INFO - qlib.Initialization - [config.py:413] - default_conf: client.
[13448:MainThread](2023-03-15 09:32:55,087) INFO - qlib.Initialization - [__init__.py:74] - qlib successfully initialized based on client settings.
[13448:MainThread](2023-03-15 09:32:55,089) INFO - qlib.Initialization - [__init__.py:76] - data_path={'__DEFAULT_FREQ': WindowsPath('C:/Users/is_li/.qlib/qlib_data/cn_data')}


## 参数设置

In [3]:
#训练时间设置
start_time = "2018-07-01" # 数据开始时间
end_time =  "2020-12-31" # 数据结束时间/测试结束时间/回测结束时间
train_end = "2019-06-30" # 训练结束时间
valid_start = "2019-07-01" # 验证开始时间
valid_end = "2020-06-30" # 验证结束时间
test_start = "2020-07-01" # 测试开始时间/回测开始时间
train_segments = {
                "train": (start_time, train_end),
                "valid": (valid_start, valid_end),
                "test": (test_start, end_time),
            }
#市场设置
market = "yunnan"
instruments = market

## 训练设置

In [5]:
handler = load_alpha158_handler(start_time,end_time,valid_end,market)
df = handler.fetch()# 因子数据表
# df.to_csv('C:/Users/is_li/Desktop/paper/github/stock on MLOps/MLOps/tools/DVC/example/data/dataset.csv')

model = load_lightGBM()#模型设置

dataset = DatasetH(handler,segments=train_segments)#数据集设置
# 存储数据
# dataset.prepare('test').to_csv('C:/Users/is_li/Desktop/paper/github/stock on MLOps/MLOps/tools/DVC/example/data/test.csv')
# dataset.prepare('valid').to_csv('C:/Users/is_li/Desktop/paper/github/stock on MLOps/MLOps/tools/DVC/example/data/valid.csv')
# dataset.prepare('train').to_csv('C:/Users/is_li/Desktop/paper/github/stock on MLOps/MLOps/tools/DVC/example/data/train.csv')

[13448:MainThread](2023-03-15 09:33:46,269) INFO - qlib.timer - [log.py:117] - Time cost: 51.028s | Loading data Done
[13448:MainThread](2023-03-15 09:33:46,280) INFO - qlib.timer - [log.py:117] - Time cost: 0.005s | DropnaLabel Done
c:\ProgramData\Anaconda3\envs\py380\lib\site-packages\qlib\data\dataset\processor.py:316: FutureWarning: Not prepending group keys to the result index of transform-like apply. In the future, the group keys will be included in the index, regardless of whether the applied function returns a like-indexed object.
To preserve the previous behavior, use

	>>> .groupby(..., group_keys=False)

To adopt the future behavior and silence this warning, use 

	>>> .groupby(..., group_keys=True)
  df[cols] = df[cols].groupby("datetime").apply(self.zscore_func)
c:\ProgramData\Anaconda3\envs\py380\lib\site-packages\qlib\data\dataset\processor.py:316: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_i

Please install necessary libs for CatBoostModel.
ModuleNotFoundError. XGBModel is skipped(optional: maybe installing xgboost can fix it).


c:\ProgramData\Anaconda3\envs\py380\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
# 模型训练
model.fit(dataset)

Training until validation scores don't improve for 50 rounds
[20]	train's l2: 0.894945	valid's l2: 0.9
[40]	train's l2: 0.894945	valid's l2: 0.9
Early stopping, best iteration is:
[1]	train's l2: 0.894945	valid's l2: 0.9


[13448:MainThread](2023-03-15 09:33:53,788) INFO - qlib.workflow - [exp.py:257] - Experiment 1 starts running ...
[13448:MainThread](2023-03-15 09:33:54,224) INFO - qlib.workflow - [recorder.py:293] - Recorder 32ec2f1a23304eca8ac2b149c18f62b8 starts running under Experiment 1 ...


## 输出

In [7]:
# 输出预测
predict = model.predict(dataset,segment = 'test')

### 回测

In [8]:
#参数设置
benchmark = 'sh000300'
port_analysis_config = {
    "executor": {
        "class": "SimulatorExecutor",
        "module_path": "qlib.backtest.executor",
        "kwargs": {
            "time_per_step": "day",
            "generate_portfolio_metrics": True,
        },
    },
    "strategy": {
        "class": "TopkDropoutStrategy",
        "module_path": "qlib.contrib.strategy.signal_strategy",
        "kwargs": {
            "model": model,
            "dataset": dataset,
            "topk": 5,
            "n_drop": 2,
        },
    },
    "backtest": {
        "start_time": test_start,
        "end_time": end_time,
        "account": 100000000,
        "benchmark": benchmark,
        "exchange_kwargs": {
            "freq": "day",
            "limit_threshold": 0.095,
            "deal_price": "close",
            "open_cost": 0,
            "close_cost": 0,
            "min_cost": 0,
        },
    },
}

In [9]:
#回测
from qlib.workflow.record_temp import SignalRecord, PortAnaRecord
from qlib.workflow import R
sr = SignalRecord(model=model,dataset=dataset)
R.save_objects(trained_model=model)
rid = R.get_recorder().id
recorder = R.get_recorder(recorder_id=rid)
sr = SignalRecord(model, dataset, recorder)
sr.generate()
par = PortAnaRecord(recorder, port_analysis_config, "day")
par.generate()

[13448:MainThread](2023-03-15 09:33:54,720) INFO - qlib.workflow - [record_temp.py:194] - Signal record 'pred.pkl' has been saved as the artifact of the Experiment 1


'The following are prediction results of the LGBModel model.'
                              score
datetime   instrument              
2020-07-01 SH600096   -1.250915e-09
           SZ000538   -1.250915e-09
           SZ000560   -1.250915e-09
           SZ000878   -1.250915e-09
           SZ002053   -1.250915e-09


[13448:MainThread](2023-03-15 09:33:54,770) INFO - qlib.backtest caller - [__init__.py:94] - Create new exchange
[13448:MainThread](2023-03-15 09:34:36,748) WARNING - qlib.BaseExecutor - [executor.py:127] - `common_infra` is not set for <qlib.backtest.executor.SimulatorExecutor object at 0x000001DF8E0B2F40>
backtest loop:   0%|          | 0/126 [00:00<?, ?it/s]c:\ProgramData\Anaconda3\envs\py380\lib\site-packages\qlib\utils\index_data.py:480: RuntimeWarning: Mean of empty slice
  return np.nanmean(self.data)
c:\ProgramData\Anaconda3\envs\py380\lib\site-packages\qlib\utils\index_data.py:480: RuntimeWarning: Mean of empty slice
  return np.nanmean(self.data)
c:\ProgramData\Anaconda3\envs\py380\lib\site-packages\qlib\utils\index_data.py:480: RuntimeWarning: Mean of empty slice
  return np.nanmean(self.data)
c:\ProgramData\Anaconda3\envs\py380\lib\site-packages\qlib\utils\index_data.py:480: RuntimeWarning: Mean of empty slice
  return np.nanmean(self.data)
c:\ProgramData\Anaconda3\envs\py3

'The following are analysis results of benchmark return(1day).'
                       risk
mean               0.001872
std                0.013497
annualized_return  0.445649
information_ratio  2.140217
max_drawdown      -0.071481
'The following are analysis results of the excess return without cost(1day).'
                       risk
mean              -0.000469
std                0.010799
annualized_return -0.111613
information_ratio -0.669963
max_drawdown      -0.166979
'The following are analysis results of the excess return with cost(1day).'
                       risk
mean              -0.000469
std                0.010799
annualized_return -0.111613
information_ratio -0.669963
max_drawdown      -0.166979


[13448:MainThread](2023-03-15 09:34:37,976) INFO - qlib.workflow - [record_temp.py:524] - Indicator analysis record 'indicator_analysis_1day.pkl' has been saved as the artifact of the Experiment 1


'The following are analysis results of indicators(1day).'
     value
ffr    1.0
pa     0.0
pos    0.0
